## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-27-20-50-43 +0000,wapo,Live updates: Shooter ID’d after 2 children ki...,https://www.washingtonpost.com/nation/2025/08/...
1,2025-08-27-20-50-43 +0000,wapo,Church shooting upends quiet neighborhood,https://www.washingtonpost.com
2,2025-08-27-20-48-34 +0000,nyt,"Trump, With Tariffs and Threats, Tries to Stro...",https://www.nytimes.com/2025/08/27/climate/tru...
3,2025-08-27-20-48-15 +0000,nyt,‘Alligator Alcatraz’ Detention Center May Be E...,https://www.nytimes.com/2025/08/27/us/alligato...
4,2025-08-27-20-48-00 +0000,wsj,Two Children Killed in Minneapolis Catholic Sc...,https://www.wsj.com/us-news/minnesota-school-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,64
198,new,16
55,gaza,16
9,school,15
378,president,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
284,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...,157
301,2025-08-26-22-42-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,148
302,2025-08-26-22-33-58 +0000,nypost,Fed governor Lisa Cook hires Hunter Biden lawy...,https://nypost.com/2025/08/26/us-news/fed-gove...,134
159,2025-08-27-12-22-00 +0000,nyt,Lisa Cook Defies Trump Bid to Fire Her as Fed ...,https://www.nytimes.com/2025/08/26/us/politics...,122
139,2025-08-27-13-50-15 +0000,nyt,Trump Is Set to Host a Meeting on Gaza. How Cl...,https://www.nytimes.com/2025/08/27/world/middl...,114


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
284,157,2025-08-27-00-11-00 +0000,wsj,President Trump’s attempt to fire Fed governor...,https://www.wsj.com/politics/policy/trumps-fed...
46,84,2025-08-27-18-34-05 +0000,nypost,Mom of Minneapolis mass shooter Robin Westman ...,https://nypost.com/2025/08/27/us-news/mom-of-m...
235,47,2025-08-27-07-04-00 +0000,wsj,Israeli Troops Targeted a Camera in Gaza Hospi...,https://www.wsj.com/world/middle-east/israeli-...
193,44,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...
172,43,2025-08-27-11-15-50 +0000,cbc,Denmark summons top U.S. diplomat after report...,https://www.cbc.ca/news/world/denmark-us-green...
273,43,2025-08-27-01-00-00 +0000,wsj,The first Black woman to serve as a Federal Re...,https://www.wsj.com/economy/central-banking/li...
138,42,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...
294,38,2025-08-26-23-05-09 +0000,nypost,"Cracker Barrel reverses course on new logo, br...",https://nypost.com/2025/08/26/business/cracker...
91,37,2025-08-27-16-24-45 +0000,nypost,Heartbreaking photos show Minneapolis Catholic...,https://nypost.com/2025/08/27/us-news/heartbre...
255,33,2025-08-27-03-46-15 +0000,nypost,Some FEMA employees put on leave following sig...,https://nypost.com/2025/08/26/us-news/some-fem...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
